In [67]:
import re

class WhileLoop:
    def __init__(self, condition, content):
        self.condition = condition
        self.content = content

# Open the C source file and read its contents into a string
with open('source.c', 'r') as f:
    source = f.read()

# Find the start and end indices of the while loop
brace_count = 1
condition_start = re.search('while\s*\(', source).end()
condition_end = source.index(')', condition_start)

content_start = source.index('{', condition_end) + 1
condition = source[condition_start:condition_end].strip()
for i, c in enumerate(source[content_start:]):
    if c == '{':
        brace_count += 1
    elif c == '}':
        brace_count -= 1
        if brace_count == 0:
            content_end = content_start + i
            break
# remove indents
content = re.sub(r'^\s+|\s+$', '', source[content_start:content_end].strip(), flags=re.MULTILINE)
print("================================")
loop = WhileLoop(condition, content)
print(loop.condition)
print(loop.content)
print("================================")


y <= y2
int x = x1;
while (x <= xR) {
setPixel(x, y, 1);
x++;
}
while (errR < 0) {
xR++;
errR = errR + 2*(y2-y1);
}
errR = errR - 2*(x2-x1);
y++;


In [9]:
def print_while_tree(tree, indent=0):
    for node in tree:
        print(" " * indent + f"while ({node.condition}) {{")
        print_while_tree(node.children, indent + 4)
        print(" " * indent + "}")

# Print the while loop tree
print_while_tree(tree)
